In [2]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [7]:
slist = ['A191RL', 'DPCERY', 'DDURRY', 'DNDGRY', 'DSERRY', 'A006RY',
         'A008RY', 'A011RY', 'A014RY', 'A822RY', 'A823RY', 'A829RY', 
         'A019RY', 'A020RY', 'A021RY']

n = {'A191RL': '\cbox{red!95!black} & \\textbf{Gross Domestic Product}',
     'DPCERY': '\cbox{yellow!80!orange} & \hspace{2mm} Consumer Spending',
     'DDURRY': '& \hspace{4mm} Durable Goods',
     'DNDGRY': '& \hspace{4mm} Non-durable Goods ',
     'DSERRY': '& \hspace{4mm} Services ',
     'A006RY': '\cbox{blue!70!black} & \hspace{2mm} Gross Investment',
     'A008RY': '& \hspace{4mm} Residential ',
     'A011RY': '& \hspace{4mm} Non-residential ',
     'A014RY': '& \hspace{4mm} Change in inventories ',
     'A019RY': '\cbox{green!60!black} & \hspace{2mm} Net Exports ',
     'A020RY': '& \hspace{4mm} Exports ',
     'A021RY': '& \hspace{4mm} Imports ',
     'A822RY': '\cbox{cyan!50!white} & \hspace{2mm} Government ',
     'A823RY': '& \hspace{4mm} Federal ',
     'A829RY': '& \hspace{4mm} State and Local '}


va = pd.read_csv(data_dir / 'gdpvafull.csv', parse_dates=['date']).set_index('date')

sl2 = ['GoodsTTU', 'Manufacturing', 'Construction', 'Retail trade', 'FIRE', 
       'Oth_Serv', 'Educational services, health care, and social assistance',
       'Professional and business services', 'Information', 'Government']

n2 = {'GoodsTTU': '\cbox{purple!70!blue} & \hspace{2mm} Goods and TTU ',
      'Manufacturing': '& \hspace{4mm} Manufacturing ',
      'Construction': '& \hspace{4mm} Construction ',
      'Retail trade': '& \hspace{4mm} Retail Trade ',
      'FIRE': '\cbox{red!90!white} & \hspace{2mm} FIRE ',
      'Oth_Serv': '\cbox{blue!90!white} & \hspace{2mm} Other Services ',
      'Educational services, health care, and social assistance': '& \hspace{4mm} Education \& Healthcare ',
      'Professional and business services': '& \hspace{4mm} Professional \& Business',
      'Information': '& \hspace{4mm} Information ',
      'Government': '\cbox{orange!80!white} & \hspace{2mm} Government '}

jb = pd.read_csv(data_dir / 'gdpjobs.csv', parse_dates=['date']).set_index('date').drop(pd.to_datetime('2019-07-01'))

n3 = {'pop_contr': '\cbox{lime} & \hspace{2mm} Population ',
      'epop_contr': '\cbox{green!30!teal!90!black} & \hspace{2mm} Employment Rate ',
      'hours_contr': '\cbox{blue} & \hspace{2mm} Average Hours',
      'prod': '\cbox{cyan!50!white} & \hspace{2mm} Productivity '}

sl3 = ['pop_contr', 'epop_contr', 'hours_contr', 'prod']

gd = pd.read_csv(data_dir / 'gdi.csv', parse_dates=['date']).set_index('date')

n4 = {'A261RC': '& \\textbf{Gross Domestic Income} ',
      'A4002C': '\cbox{magenta!90!blue} & \hspace{2mm} Labor ',
      'W271RC': '\cbox{yellow!60!orange} & \hspace{2mm} Profit ',
      'A262RC': '\cbox{teal!60!white} & \hspace{2mm} Depreciation ',
      'indirect': '\cbox{violet} & \hspace{2mm} Indirect Taxes '}

sl4 = ['A261RC', 'A4002C', 'W271RC', 'A262RC', 'indirect']

In [8]:
df = nipa_df(retrieve_table('T10502')['Data'], slist)

In [9]:
#nipa_series_codes(retrieve_table('T10502'))

In [10]:
# GDP Table (please consider revising!!)
f = data_dir / 'gdptable.tex'
os.remove(f)

date1 = {}
date = {}
data = {s: {} for s in list(df.keys())}

blankrow = '& & & & & & & & & & \\\\'

with open(f, 'a') as table:
    
    for s in list(df.keys()):
        data[s]['name'] = n[s]

    for i in range(1, 6):
        dt = df['A191RL'].index[-i]
        q = dt.quarter
        y = dt.year
        date1[i] = dt
        if i == 1:
            date[i] = f'{y} Q{q}'
        else:
            date[i] = f"'{str(y)[-2:]} Q{q}"
        for s in list(df.keys()):
            if s == 'A191RL':
                data[s][i] = f'{df[s].iloc[-i]:.1f}'
            else:
                data[s][i] = f'{df[s].iloc[-i]:.2f}'

    for ma in [13, 41, 121]:
        for s in list(df.keys()):
            if s == 'A191RL':
                data[s][ma] = f'{df[s].iloc[-ma:].mean():.1f}'
            else:
                data[s][ma] = f'{df[s].iloc[-ma:].mean():.2f}'

    daterow = (f'& & {date[1]} & {date[2]} & {date[3]} & {date[4]} & {date[5]}'
                ' & & 3-year & 10-year & 30-year \\\\')
    table.write(daterow + '\n')
    print(daterow)

    for s in slist:
        r = (f'{n[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')

        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow + '\n')

    for s in list(va.keys()):
        data[s] = {}
        for i in range(1, 6):
            if date1[i] in va.index:
                data[s][i] = f'{va[s].loc[date1[i]]:.2f}'
            else:
                data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(va.keys()):
            data[s][ma] = f'{va[s].iloc[-ma:].mean():.2f}'

    for s in sl2:
        r = (f'{n2[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow + '\n')

    for s in list(jb.keys()):
        data[s] = {}
        for i in range(1, 6):
            if date1[i] in jb.index:
                data[s][i] = f'{jb[s].loc[date1[i]]:.2f}'
            else:
                data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(jb.keys()):
            data[s][ma] = f'{jb[s].iloc[-ma:].mean():.2f}'

    for s in sl3:
        r = (f'{n3[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow)

    for s in list(gd.keys()):
        data[s] = {}
        if s == 'A261RC':
            for i in range(1, 6):
                if date1[i] in gd.index:
                    data[s][i] = f'{gd[s].loc[date1[i]]:.1f}'
                else:
                    data[s][i] = '--'
        else:
            for i in range(1, 6):
                if date1[i] in gd.index:
                    data[s][i] = f'{gd[s].loc[date1[i]]:.2f}'
                else:
                    data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(gd.keys()):
            if s == 'A261RC':
                data[s][ma] = f'{gd[s].iloc[-ma:].mean():.1f}'
            else:
                data[s][ma] = f'{gd[s].iloc[-ma:].mean():.2f}'

    for s in sl4:
        r = (f'{n4[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        print(r)
        table.write(r + '\n')


& & 2019 Q2 & '19 Q1 & '18 Q4 & '18 Q3 & '18 Q2 & & 3-year & 10-year & 30-year \\
\cbox{red!95!black} & \textbf{Gross Domestic Product} & 2.0 & 3.1 & 1.1 & 2.9 & 3.5 & & 2.5 &  2.2 & 2.5 \\
\cbox{yellow!80!orange} & \hspace{2mm} Consumer Spending & 3.10 & 0.78 & 0.97 & 2.34 & 2.70 & & 1.88 &  1.58 & 1.81 \\
& \hspace{4mm} Durable Goods & 0.87 & 0.02 & 0.09 & 0.25 & 0.56 & & 0.44 &  0.42 & 0.42 \\
& \hspace{4mm} Non-durable Goods  & 0.91 & 0.30 & 0.24 & 0.50 & 0.57 & & 0.40 &  0.30 & 0.33 \\
& \hspace{4mm} Services  & 1.32 & 0.46 & 0.65 & 1.59 & 1.57 & & 1.03 &  0.85 & 1.06 \\
\cbox{blue!70!black} & \hspace{2mm} Gross Investment & -1.11 & 1.09 & 0.53 & 2.27 & -0.30 & & 0.62 &  0.87 & 0.59 \\
& \hspace{4mm} Residential  & -0.09 & 0.60 & 0.64 & 0.29 & 1.04 & & 0.60 &  0.58 & 0.54 \\
& \hspace{4mm} Non-residential  & -0.11 & -0.04 & -0.18 & -0.16 & -0.15 & & -0.01 &  0.11 & 0.03 \\
& \hspace{4mm} Change in inventories  & -0.91 & 0.53 & 0.07 & 2.14 & -1.20 & & 0.04 &  0.17 & 0.03 \\
\cbox{c

### Consumer Spending and Investment

In [13]:
s = ['DNPIRY', 'DSERRY', 'DPCERY', 'DGDSRY', 'DMOTRY',
     'DFDHRY', 'DREQRY', 'DFXARY', 'DCLORY', 'DHLCRY',
     'DTRSRY', 'DRCARY', 'DFSARY', 'DIFSRY', 'DHUTRY',
     'A011RY']

n = {'TOTAL': 'Total',
     'DGDSRY': '\cbox{yellow!80!orange} & \hspace{2mm} Goods',
     'DMOTRY': '& \hspace{4mm} Motor Vehicles and Parts',
     'DFDHRY': '& \hspace{4mm} Furniture and HH Equipment',
     'DREQRY': '& \hspace{4mm} Recreational Vehicles and Goods',
     'DFXARY': '& \hspace{4mm} Groceries',
     'DCLORY': '& \hspace{4mm} Clothes and Shoes',
     'OTHSERV': '\cbox{blue!70!black} & \hspace{2mm} Services',
     'DHLCRY': '& \hspace{4mm} Health Care Services',
     'DTRSRY': '& \hspace{4mm} Transportation',
     'DRCARY': '& \hspace{4mm} Recreational',
     'DFSARY': '& \hspace{4mm} Food and Accommodations',
     'DIFSRY': '& \hspace{4mm} Financial and Insurance',
     'SHELTER': '\cbox{green!60!black} & \hspace{2mm} Shelter ',
     'DHUTRY': '& \hspace{4mm} Housing Services and Utilities ',
     'A011RY': '& \hspace{4mm} Private Residential Fixed Investment'}


total = lambda x: x['DPCERY'] - x['DNPIRY']
othserv = lambda x: x['DSERRY'] - x['DNPIRY'] - x['DHUTRY']
shelter = lambda x: x['DHUTRY'] + x['A011RY']

In [18]:
(nipa_df(retrieve_table('T10502')['Data'], s)
 .assign(TOTAL = total, OTHSERV = othserv, SHELTER = shelter)
 [list(n.keys())])

,TOTAL,DGDSRY,DMOTRY,DFDHRY,DREQRY,DFXARY,DCLORY,OTHSERV,DHLCRY,DTRSRY,DRCARY,DFSARY,DIFSRY,SHELTER,DHUTRY,A011RY
1989-01-01,1.14,0.15,-0.01,0.15,-0.23,0.11,0.04,0.72,0.08,-0.02,0.12,-0.07,0.49,0.08,0.27,-0.19
1989-04-01,0.95,0.57,0.04,0.13,0.25,-0.16,0.18,0.32,-0.11,0.00,0.07,-0.04,0.19,-0.54,0.05,-0.59
1989-07-01,2.21,1.46,0.67,-0.04,0.14,0.18,0.30,0.46,0.01,0.02,0.06,0.15,0.06,0.21,0.29,-0.08
1989-10-01,0.89,-0.33,-1.21,0.02,0.02,0.23,0.05,0.47,0.31,-0.02,0.01,0.06,0.04,0.37,0.75,-0.38
1990-01-01,2.04,1.36,1.23,0.12,0.07,0.03,-0.11,1.19,0.41,0.02,0.15,0.39,0.05,-0.35,-0.51,0.16
1990-04-01,0.42,-1.09,-0.75,-0.23,-0.04,0.40,-0.20,0.87,0.40,0.02,0.14,0.14,0.02,-0.05,0.64,-0.69
1990-07-01,0.63,-0.24,-0.22,-0.11,-0.02,0.01,-0.03,0.53,0.31,-0.03,0.03,0.12,0.10,-0.56,0.33,-0.89
1990-10-01,-2.11,-1.58,-0.75,-0.09,-0.02,-0.15,-0.16,-0.61,0.08,-0.09,0.04,-0.25,0.13,-0.82,0.08,-0.90
1991-01-01,-1.09,-0.87,-0.93,0.00,0.09,-0.06,0.14,-0.25,-0.03,-0.28,-0.15,-0.14,0.64,-0.75,0.03,-0.78
1991-04-01,2.03,0.54,-0.08,0.08,0.15,0.11,0.19,0.81,0.21,-0.05,-0.04,0.34,0.43,1.11,0.68,0.43


In [ ]:
'DRCARY': 'Recreation services',
 'DFSARY': 'Food services and accommodations',
 'DIFSRY'